In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import f1_score, classification_report
import gc # 垃圾回收
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter # 用于检查类别分布
# ---------------------# 💡 新增：导入 RandomUnderSampler# -----------------------
from imblearn.under_sampling import RandomUnderSampler

# 设置 Matplotlib/Seaborn 样式
sns.set(style='whitegrid')
# 设置 Pandas 显示选项
pd.set_option('display.max_columns', None)

In [2]:
print("正在加载特征化训练数据 (train_with_features.csv)...")
try:
    df_train = pd.read_csv('train_with_features.csv', low_memory=False)
    print(f"数据加载成功！形状: {df_train.shape}")
except FileNotFoundError:
    print("错误：train_with_features.csv 文件未找到。")
    raise# 恢复数据类型
print("正在恢复数据类型...")
df_train['t'] = pd.to_datetime(df_train['t'])
df_train['ticker_id'] = df_train['ticker_id'].astype('category')
print("数据类型恢复完毕。")

正在加载特征化训练数据 (train_with_features.csv)...
数据加载成功！形状: (1932, 68736)
正在恢复数据类型...
数据类型恢复完毕。


In [3]:
# 1. 定义 5 分类目标映射
target_map = {'HH': 0, 'HL': 1, 'LH': 2, 'LL': 3}
# 'None' (即 NaN) 映射为 4
df_train['class_label_encoded'] = df_train['class_label'].map(target_map)
df_train['class_label_encoded'] = df_train['class_label_encoded'].fillna(4)
df_train['class_label_encoded'] = df_train['class_label_encoded'].astype(int)

print("目标变量编码完成。")
print(df_train['class_label_encoded'].value_counts().sort_index())

目标变量编码完成。
class_label_encoded
0      40
1      28
2      18
3      26
4    1820
Name: count, dtype: int64


In [4]:
# 1. 定义目标变量
TARGET = 'class_label_encoded'# 2. 定义元数据列
METADATA_COLS = ['class_label', 'class_label_encoded', 't', 'id', 'train_id']

# 3. 定义特征列 (使用所有特征)
features_to_drop = [col for col in METADATA_COLS if col in df_train.columns]
FEATURES = df_train.columns.drop(features_to_drop).tolist()

print(f"Model v4 将使用全部 {len(FEATURES)} 个特征。")
if 'ticker_id' in FEATURES:
    print("'ticker_id' 已正确包含在特征中。")

Model v4 将使用全部 68732 个特征。
'ticker_id' 已正确包含在特征中。


In [5]:
# 1. 确保数据严格按时间排序
print("正在按 't' 和 'ticker_id' 排序数据以进行验证分割...")
df_train = df_train.sort_values(by=['t', 'ticker_id']).reset_index(drop=True)

# 2. 定义 80/20 分割点
split_percentage = 0.8
split_index = int(len(df_train) * split_percentage)

# 3. 分割数据
train_data = df_train.iloc[:split_index]
val_data = df_train.iloc[split_index:]
print(f"新训练集样本数: {len(train_data)}, 新验证集样本数: {len(val_data)}")

# 4. 创建 X (特征) 和 y (目标)
X_train = train_data[FEATURES]
y_train = train_data[TARGET]
X_val = val_data[FEATURES]
y_val = val_data[TARGET]

# 5. 修复数据类型 (与 V3 中修复错误时相同)
print("正在修复 'object' 数据类型...")
numeric_features = [col for col in FEATURES if col != 'ticker_id']
X_train_copy = X_train.copy()
X_val_copy = X_val.copy()
for col in numeric_features:
    X_train_copy[col] = pd.to_numeric(X_train_copy[col], errors='coerce', downcast='float')
    X_val_copy[col] = pd.to_numeric(X_val_copy[col], errors='coerce', downcast='float')
X_train = X_train_copy
X_val = X_val_copy
print(f"已将 {len(numeric_features)} 个特征列强制转换为数值类型。")

# 6. 清理内存
del df_train, train_data, val_data, X_train_copy, X_val_copy
gc.collect()
print("验证集创建完毕！")

正在按 't' 和 'ticker_id' 排序数据以进行验证分割...
新训练集样本数: 1545, 新验证集样本数: 387
正在修复 'object' 数据类型...
已将 68731 个特征列强制转换为数值类型。
验证集创建完毕！


In [6]:
print("--- 原始训练集 (X_train) 分布 ---")
original_counts = Counter(y_train)
print(original_counts)

# 1. 定义我们的策略
# 我们要保留所有的稀有类别 (0, 1, 2, 3)
# 我们要把 'None' (类别 4) 的数量减少到与所有稀有类别 *总和* 相等
sum_minorities = original_counts[0] + original_counts[1] + original_counts[2] + original_counts[3]

# 创建策略字典
sampling_strategy = {
    0: original_counts[0],  # 保留所有 'HH'
    1: original_counts[1],  # 保留所有 'HL'
    2: original_counts[2],  # 保留所有 'LH'
    3: original_counts[3],  # 保留所有 'LL'
    4: sum_minorities       # 💡 将 'None' 的数量减少到与稀有类总和相等
}

print(f"\n--- 欠采样策略 ---")
print(f"稀有类总数: {sum_minorities}")
print(f"目标策略: {sampling_strategy}")

# 2. 初始化并应用采样器
# random_state=42 确保每次运行结果都一样
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)

print("\n正在应用 RandomUnderSampler (只对训练集)...")
X_train_sampled, y_train_sampled = rus.fit_resample(X_train, y_train)

# 3. 检查新的平衡后分布
print("\n--- 新的 (采样后) 训练集分布 ---")
print(Counter(y_train_sampled))
print(f"原始 X_train 形状: {X_train.shape}")
print(f"采样后 X_train_sampled 形状: {X_train_sampled.shape}")

--- 原始训练集 (X_train) 分布 ---
Counter({4: 1461, 0: 30, 3: 21, 1: 19, 2: 14})

--- 欠采样策略 ---
稀有类总数: 84
目标策略: {0: 30, 1: 19, 2: 14, 3: 21, 4: 84}

正在应用 RandomUnderSampler (只对训练集)...

--- 新的 (采样后) 训练集分布 ---
Counter({4: 84, 0: 30, 3: 21, 1: 19, 2: 14})
原始 X_train 形状: (1545, 68732)
采样后 X_train_sampled 形状: (168, 68732)


In [7]:
# 1. 定义 LightGBM 模型参数
lgb_params = {
    'objective': 'multiclass',
    'num_class': 5,
    'metric': 'multi_logloss',
    'n_estimators': 500,
    'learning_rate': 0.05,
    'n_jobs': -1,
    'seed': 42,
    
    # --- 💡 关键修改 💡 ---
    # 我们已经手动平衡了数据，不再需要这个参数
    # 'class_weight': 'balanced'  # <--- 已移除!
}

# 2. 初始化模型
model_v4 = lgb.LGBMClassifier(**lgb_params)

# 3. 训练模型
print(f"开始训练 Model v4 (使用采样后数据)...")
model_v4.fit(
    X_train_sampled, y_train_sampled,  # 👈 使用采样后的数据
    eval_set=[(X_val, y_val)],          # 👈 评估仍使用 *原始* 验证集
    eval_metric='multi_logloss',
    callbacks=[lgb.early_stopping(50)],
    categorical_feature=['ticker_id']
)

print("Model v4 训练完成！")

开始训练 Model v4 (使用采样后数据)...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000822 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3131
[LightGBM] [Info] Number of data points in the train set: 168, number of used features: 141
[LightGBM] [Info] Start training from score -1.722767
[LightGBM] [Info] Start training from score -2.179525
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't i

In [8]:
# 1. 在验证集上进行预测
print("正在验证集上进行预测...")
val_preds = model_v4.predict(X_val)

# 2. 计算 Macro F1-Score
macro_f1_v4 = f1_score(y_val, val_preds, average='macro', zero_division=0)

print("\n========================================")
print(f"📈 Model v4 (Undersampled) Macro F1-Score: {macro_f1_v4:.4f}")
print("----------------------------------------")
print(f"📈 Model v2 (All Features) Macro F1-Score: 0.2238")
print(f"📉 Model v3 (Top 300) Macro F1-Score: 0.1916")
print(f"📉 Baseline Macro F1-Score: 0.1905")
print("========================================")
print(f"🚀 V4 vs V2 (Best) 性能提升: {macro_f1_v4 - 0.2238:.4f}")
print("========================================")

# 3. 打印详细的分类报告
target_map_full = {'HH': 0, 'HL': 1, 'LH': 2, 'LL': 3, 'None': 4}
target_names = [key for key, val in sorted(target_map_full.items(), key=lambda item: item[1])]
all_labels = [val for key, val in sorted(target_map_full.items(), key=lambda item: item[1])]

print("\n详细分类报告 (Model v4):")
print(classification_report(
    y_val, 
    val_preds, 
    target_names=target_names, 
    labels=all_labels,
    zero_division=0
))

正在验证集上进行预测...

📈 Model v4 (Undersampled) Macro F1-Score: 0.2250
----------------------------------------
📈 Model v2 (All Features) Macro F1-Score: 0.2238
📉 Model v3 (Top 300) Macro F1-Score: 0.1916
📉 Baseline Macro F1-Score: 0.1905
🚀 V4 vs V2 (Best) 性能提升: 0.0012

详细分类报告 (Model v4):
              precision    recall  f1-score   support

          HH       0.04      0.10      0.05        10
          HL       0.00      0.00      0.00         9
          LH       0.07      0.25      0.11         4
          LL       0.06      0.40      0.11         5
        None       0.93      0.78      0.85       359

    accuracy                           0.74       387
   macro avg       0.22      0.31      0.23       387
weighted avg       0.87      0.74      0.79       387



In [2]:
!pip install imbalanced-learn -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 1.6 MB/s eta 0:00:0000:0100:01
